In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-dataset/sentiment_data.csv


In [41]:
#loading libraries
! pip install tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from tqdm import tqdm
from textblob import TextBlob
import time
import re
import gensim
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
tqdm.pandas()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/sentiment-analysis-dataset/sentiment_data.csv


In [79]:
df = pd.read_csv('/kaggle/input/sentiment-analysis-dataset/sentiment_data.csv')
df.head()

,Unnamed: 0,Comment,Sentiment
0,0,lets forget apple pay required brand new iphon...,1
1,1,nz retailers don’t even contactless credit car...,0
2,2,forever acknowledge channel help lessons ideas...,2
3,3,whenever go place doesn’t take apple pay doesn...,0
4,4,apple pay convenient secure easy use used kore...,2


In [60]:
df['Comment'][0]

'lets forget apple pay required brand new iphone order use significant portion apples user base wasnt able use even wanted successive iphone incorporated technology older iphones replaced number people could use technology increased'

In [28]:
df.dtypes

Unnamed: 0     int64
Comment       object
Sentiment      int64
dtype: object

In [80]:
df['Comment'].dropna()

0         lets forget apple pay required brand new iphon...
1         nz retailers don’t even contactless credit car...
2         forever acknowledge channel help lessons ideas...
3         whenever go place doesn’t take apple pay doesn...
4         apple pay convenient secure easy use used kore...
                                ...                        
241140    crores paid neerav modi recovered congress lea...
241141    dear rss terrorist payal gawar modi killing pl...
241142                         cover interaction forum left
241143    big project came india modi dream project happ...
241144    ever listen like gurukul discipline maintained...
Name: Comment, Length: 240928, dtype: object

In [62]:
df.shape

(241145, 3)

In [81]:
#Cleaning Data
def proper_text(text):
    if isinstance(text, str):  # Make sure the message is text
        text = text.lower() #Converts to lower case
        text = re.sub(r'[^\w\s]', '',text) #Removes puncutation
        text = re.sub(r'\d+','',text) # Removes digits
    else:
        return ' ' 
    return text
df['Clean_data'] = df['Comment'].progress_apply(proper_text)    

100%|██████████| 241145/241145 [00:01<00:00, 162956.94it/s]


In [82]:
df.head()

,Unnamed: 0,Comment,Sentiment,Clean_data
0,0,lets forget apple pay required brand new iphon...,1,lets forget apple pay required brand new iphon...
1,1,nz retailers don’t even contactless credit car...,0,nz retailers dont even contactless credit card...
2,2,forever acknowledge channel help lessons ideas...,2,forever acknowledge channel help lessons ideas...
3,3,whenever go place doesn’t take apple pay doesn...,0,whenever go place doesnt take apple pay doesnt...
4,4,apple pay convenient secure easy use used kore...,2,apple pay convenient secure easy use used kore...


In [83]:
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
def tokenize(text):
    if isinstance(text, str): 
        return tokenizer.tokenize(text)
    else:
        return ' '

In [84]:
df['Clean_data'] = df['Clean_data'].progress_apply(tokenize)  

100%|██████████| 241145/241145 [00:01<00:00, 139270.63it/s]


In [85]:
df.head()

,Unnamed: 0,Comment,Sentiment,Clean_data
0,0,lets forget apple pay required brand new iphon...,1,"[lets, forget, apple, pay, required, brand, ne..."
1,1,nz retailers don’t even contactless credit car...,0,"[nz, retailers, dont, even, contactless, credi..."
2,2,forever acknowledge channel help lessons ideas...,2,"[forever, acknowledge, channel, help, lessons,..."
3,3,whenever go place doesn’t take apple pay doesn...,0,"[whenever, go, place, doesnt, take, apple, pay..."
4,4,apple pay convenient secure easy use used kore...,2,"[apple, pay, convenient, secure, easy, use, us..."


In [89]:
vocab = []
for i in df.Clean_data:
    for j in i:
        vocab.append(j)

In [90]:
vocab[0:10]

['lets',
 'forget',
 'apple',
 'pay',
 'required',
 'brand',
 'new',
 'iphone',
 'order',
 'use']

In [92]:
total_words = len(list(set(vocab)))
total_words

155381

In [108]:
max_len = -1
for doc in df.Comment:
    token = nltk.word_tokenize(doc)
    if(max_len<len(token)):
        max_len = len(token)
  

TypeError: expected string or bytes-like object, got 'float'

In [107]:
print(max_len)

-1


In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(df['Clean_data'])
sequence = tokenizer.texts_to_sequences(df['Clean_data'])

X = pad_sequences(sequence, maxlen=max_len)
y = df['Sentiment'].values

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)